In [2]:
import os
import ast
import sys
src_dir = os.path.join('..', 'src')
sys.path.append(os.path.abspath(src_dir))

import requests
import json
from os import path as osp
from datetime import datetime, timedelta
import praw

from security import KeyHandler
from data import path

VERSION = 'PedAPI/0.0.3'


class RedditClient:
    def __init__(self, verbose: bool = True):
        self.verbose_print = print if verbose else lambda *a, **k: None
        self.save_path = path("comments")

        self.session_time_out = datetime.now()
        self.kh = KeyHandler()
        self._login()
        self.save_path = path("comments")

    def _login(self):
        authorization = requests.auth.HTTPBasicAuth(
            self.kh.client_id,
            self.kh.secret_key)

        self.kh.headers['User-Agent'] = VERSION
        res = requests.post('https://www.reddit.com/api/v1/access_token',
                            auth=authorization,
                            data=self.kh.login_data,
                            headers=self.kh.headers)
        self.session_time_out = datetime.now() + timedelta(seconds=res.json()['expires_in'] - 60)

        token = res.json()['access_token']
        self.kh.headers['Authorization'] = f'bearer {token}'

        assert self._test_authentication()
        self.verbose_print('Successfully logged as {}'.format(self.kh.login_data['username']))
        self.verbose_print('Session expires on {}'.format(str(self.session_time_out)))

    def _test_authentication(self):
        if requests.get('https://oautch.reddit.com/api/v1/me', headers=self.kh.headers).status_code == 200:
            return True
        return False

    def _manage_session_time_out(self):
        if datetime.now() > self.session_time_out:
            self.verbose_print('Renewing session...')
            self._login()

    def get_comments(self, post_id: str):
        self._manage_session_time_out()

        link = 'https://oauth.reddit.com/r/wallstreetbets/comments/'
        res = requests.get(link + post_id, headers=self.kh.headers)

        with open(osp.join(self.save_path, post_id + '.json'), 'w') as outfile:
            json.dump(res.json(), outfile, indent=4)


class Praw:
    def __init__(self, verbose: bool = True):
        self.verbose_print = print if verbose else lambda *a, **k: None

        self.kh = KeyHandler()
        self._login()

    def _login(self):
        self.reddit = praw.Reddit(
            client_id=self.kh.client_id,
            client_secret=self.kh.secret_key,
            user_agent=VERSION,
            username=self.kh.login_data['username'],
            password=self.kh.login_data['password']
        )

    def get_submission(self, id_):
        return self.reddit.submission(id=id_)


In [3]:
client = Praw()

In [4]:
submission = client.get_submission('nrblju')

In [9]:
for i, top_level_comment in enumerate(submission.comments):
    while True:
        try:
            submission.comments.replace_more(None)
            break
        except:
            print("Handling replace_more exception")
            sleep(1)
            
    print(i, top_level_comment.body, top_level_comment.created_utc)

0 Main mods I'm using are More Archotech Garbage, Science never stops AOTC, Rimatomics, Rimfactory, Misc Robots, VFE mechs plus a bunch of OP enemy factions to try (and fail) to balance the AOTC stuff a bit.

I've retextured the Cosmic reactors from AOTC as well to fit in more with the Archotech style of the base.  


Also I wouldn't suggest doing a circle style base, it is ridiculously resource and work intensive, does look pretty cool though. 1622721313.0
1 Ah, yes, the Arcotech and glitter tech "hell hole". Let's be clear, if you offer me a ride off of this base, I'll stay on the base. Pretty sure it's self sufficient at this point. 1622728569.0
2 I don't get how people can make such aesthetic bases. It's utilitarian rectangles all the way for me... 1622740894.0
3 A circular kill box... intriguing. 1622722668.0
4 Round AND mountain? Now this is quite the rare sight, most mountain bases on here are boxy and aren’t such an interesting shape. Good job. 1622722889.0
5 Off to crash somew

In [10]:
submission.num_comments

53

In [11]:
wanted = set(['all_awardings', 'body', 'created', 'created_utc', 'depth', 'downs', 'id', 'parent_id', 'replies', 'score', 'ups', 'data', 'kind', 'children', 'name'])

In [60]:
import pandas as pd
import numpy as np

def replace_more(submission):
    while True:
        try:
            submission.comments.replace_more()
            break
        except:
            print("Handling replace_more exception")
            sleep(1)

def obtain_comments_for_id(praw_client, submission_id):
    submission = praw_client.get_submission(submission_id)

    replace_more(submission)
    
    res = [obtain_comments(comment, submission_id) for comment in list(submission.comments)]
    df = pd.concat(res, ignore_index=True)
    df.columns = ['id', 'parent_id', 'post_id', 'body', 'created_utc', 'depth', 'score', 'ups', 'downs']
    return df
    

def obtain_comments(comment, submission_id):

    item = [comment.id, comment.parent_id[3:], submission_id, comment.body, comment.created_utc, comment.depth, comment.score, comment.ups, comment.downs]
        
    replies = []
    for reply in list(comment.replies):
        replies.extend([obtain_comments(reply, submission_id)])
    
    return pd.concat([pd.DataFrame(np.array([item]))] + replies)

In [61]:
obtain_comments_for_id(client, 'nrblju')

,id,parent_id,post_id,body,created_utc,depth,score,ups,downs
0,h0fg6t4,nrblju,nrblju,Main mods I'm using are More Archotech Garbage...,1622721313.0,0,44,44,0
1,h0gj95t,h0fg6t4,nrblju,"Why circles? Only my warehouses, stables and g...",1622738934.0,1,10,10,0
2,h0h7rr1,h0gj95t,nrblju,I've just done a lot of square boxy bases with...,1622749153.0,2,14,14,0
3,h0hzba0,h0gj95t,nrblju,Why corridors 3 tiles wide? I never got proble...,1622761158.0,2,3,3,0
4,h0hztbs,h0hzba0,nrblju,You can cram more defenders in wider corridors...,1622761403.0,3,4,4,0
5,h0in1y6,h0hztbs,nrblju,I always tell myself that it's for practical r...,1622772776.0,4,3,3,0
6,h0jmvl4,h0in1y6,nrblju,Isnt walking though 1/2-wide corridor slower?,1622795687.0,5,3,3,0
7,h0k0866,h0hztbs,nrblju,"But they can get more attackers in, and usuall...",1622806640.0,4,1,1,0
8,h0kb7wq,h0k0866,nrblju,You can put melee-blocking chokepoints into 3-...,1622812531.0,5,2,2,0
9,h0k6c63,h0k0866,nrblju,"It’s from over function really, there are defi...",1622810024.0,5,1,1,0


In [64]:
def identity(x: str) -> str:
    return x

In [65]:
identity(12)

12